In [25]:
import numpy as np
import onnx
import onnxruntime as ort
import time
import os
from tensorflow.nn import convolution
import matplotlib.pyplot as plt
import csv

class Multiply_experiment_class():
    #TODO: Set the input size to the size of a layer in the TINYYOLO 
    def __init__(self):
        self.in1 = np.random.random([1,3,512,512]).astype(np.float32)
        self.w = np.load("W_tensor.npy")
        self.out_ort = np.random.random([1,16,256,256]).astype(np.float32)
        self.expected_res = np.zeros_like(self.out_ort)        
        
        self.ort_runtime=[]
        
    def result_check(self):
        self.expected_res = convolution(self.in1,self.w)
        return np.allclose(self.out_ort[0],self.expected_res)
        
    def test_ort(self, model_name):
        model_load = onnx.load(model_name)
        input_dict = {"in1": self.in1, "W": self.w}
        sess = ort.InferenceSession(model_load.SerializeToString())
        self.out_ort = sess.run(None,input_dict)           
    
    def throughput_test(self,interations,model):
        for i in range(interations):
            start = time.time()
            self.test_ort(model)
            end = time.time()
            self.ort_runtime.append(end-start)       
        # return self.ort_runtime
    
    def results_publisher(self, filename = "ort_results.csv"):
        print("Interations = %d" %(len(self.ort_runtime)))
        print("Min = %f ms" %(np.mean(self.ort_runtime)*1000))
        print("Max = %f ms" %(np.max(self.ort_runtime)*1000))
        print("Mean = %f ms" %(np.min(self.ort_runtime)*1000))        
        if not os.path.exists("results/"):
            os.makedirs("results/")
        np.savetxt("results/" + filename , np.asarray(self.ort_runtime), delimiter=',')
        
    def run(self, model ,iterations=100):
        
        self.test_ort(model)
        self.throughput_test(iterations,model)
        # if self.result_check():
        self.results_publisher()

In [26]:
x = Multiply_experiment_class()
x.run(model="conv_model.onnx")

Interations = 100
Min = 16.446030 ms
Max = 34.906626 ms
Mean = 13.961554 ms
